In [ ]:
from awpy.parser import DemoParser
from awpy.visualization.plot import plot_round
from dataclasses import dataclass, field, fields
from dataclasses_json import dataclass_json
import csv

In [ ]:
demo_file_name = r"demos/cloud9-vs-outsiders-m1-dust2.dem"

p = DemoParser(demofile=demo_file_name, parse_rate=128, parse_frames=True)

data = p.parse()

In [ ]:
@dataclass_json
@dataclass
class Position:
    """
    x, y, and z coordinates in a CSGO map.
    """
    x: float
    y: float
    z: float

@dataclass_json
@dataclass
class PlayerInfo:
    """
    Contains information about a CSGO player including their name, their team, and a list of their positions throughout the gme
    """
    name: str
    team: str
    positions: list[Position] # Length of this list is the number of frames in the round

@dataclass_json
@dataclass
class RoundInfo:
    """
    Contains a list of PlayerInfo objects, one for each player in the CSGO game round.
    """
    players: list[PlayerInfo] = field(default_factory=list)# Length of this list is the number of rounds in the game

@dataclass_json
@dataclass
class Game:
    """
    Contains a list of RoundInfo objects, one for each round in the CSGO game.
    """
    round_info: list[RoundInfo] = field(default_factory=list) # Length of this list is the number of rounds in the game

    def add_player_info(self, round_index: int, player_name: str, team:str, position: Position):
        """
        Uses the given information to construct a PlayerInfo object and add it to the RoundInfo object indexed by round_index
        """
        while len(self.round_info) <= round_index:
            self.round_info.append(RoundInfo())
        for player in self.round_info[round_index].players:
            if player.name == player_name:
                player.positions.append(position)
                return
        self.round_info[round_index].players.append(PlayerInfo(player_name, team, [position]))
        return

game = Game()

@dataclass_json
@dataclass
class PlayerState:
    """
    """
    round_index: int
    player_name: str
    team: str
    x: float
    y: float
    z: float

flat_game_info: list[PlayerState] = []

for game_round_index in range(0, len(data["gameRounds"])):
    frames = data["gameRounds"][game_round_index]["frames"]
    # print("Round: ", game_round_index + 1)
    for index, frame in enumerate(frames):
        # print("Frame: ", index + 1)
        for team in ["ct", "t"]:
            for player in frame[team]["players"]:
                position = Position(player["x"], player["y"], player["z"])
                # print(player["name"], position)
                game.add_player_info(game_round_index, player["name"], team, position)
                flat_game_info.append(PlayerState(game_round_index, player["name"], team, player["x"], player["y"], player["z"]))

def debug_print_game(game: Game):
    print("Round count: ", len(game.round_info))
    for round in game.round_info:
        print("Player count: ", len(round.players))
        for player in round.players:
            print("Frame count: ", len(player.positions))

# print(game.to_json())
print(flat_game_info)



In [ ]:
with open("data.csv", "w") as output_file:
    writer = csv.writer(output_file)
    writer.writerow([class_field.name for class_field in fields(PlayerState)])
    for state_entry in flat_game_info:
        writer.writerow([getattr(state_entry, field_name) for field_name in [class_field.name for class_field in fields(PlayerState)]])

In [ ]:
d = data
plot_round("best_round_ever.gif", d["gameRounds"][7]["frames"], map_name=d["mapName"], map_type="simpleradar", dark=False)

In [2]:
import parsing

demo_file_name = r"D:/Samihan/Documents/Programming/Python Projects/CSGO Demo Parsing/demos/cloud9-vs-outsiders-m1-dust2.dem"
print(f"Parsing {demo_file_name}")
data = parsing.parse_demo_file(demo_file_name)
print(data.keys())

dict_keys(['matchID', 'clientName', 'mapName', 'tickRate', 'playbackTicks', 'playbackFramesCount', 'parsedToFrameIdx', 'parserParameters', 'serverVars', 'matchPhases', 'matchmakingRanks', 'playerConnections', 'gameRounds'])


In [4]:
parser_params = data["parserParameters"]
for key in parser_params.keys():
    print(f"{key}: {parser_params[key]}")

parseRate: 128
parseFrames: False
parseKillFrames: False
tradeTime: 5
roundBuyStyle: hltv
damagesRolledUp: False


In [5]:
parser_params = data["serverVars"]
for key in parser_params.keys():
    print(f"{key}: {parser_params[key]}")

cashBombDefused: 0
cashBombPlanted: 0
cashTeamTWinBomb: 0
cashWinDefuse: 3500
cashWinTimeRunOut: 0
cashWinElimination: 0
cashPlayerKilledDefault: 0
cashTeamLoserBonus: 0
cashTeamLoserBonusConsecutive: 0
roundTime: 0
roundTimeDefuse: 0
roundRestartDelay: 5
freezeTime: 20
buyTime: 20
bombTimer: 0
maxRounds: 30
timeoutsAllowed: 4
coachingAllowed: 1


In [6]:
parser_params = data["matchPhases"]
for key in parser_params.keys():
    print(f"{key}: {parser_params[key]}")

announcementLastRoundHalf: [233488]
announcementFinalRound: []
announcementMatchStarted: []
roundStarted: [14830, 27154, 46544, 58496, 76416, 97677, 109787, 130834, 150026, 165973, 180737, 201878, 219050, 233488, 250588, 288064, 298489, 315460, 333380, 350945, 358510]
roundEnded: [14189, 26514, 45904, 57857, 75776, 97037, 109147, 130194, 149386, 165333, 180098, 201236, 218410, 232849, 250332, 287424, 297849, 314820, 332740, 350305, 357871, 370897]
roundFreezetimeEnded: [2521, 17390, 29714, 49104, 61056, 83864, 100237, 116620, 133394, 152586, 168533, 188188, 204438, 221610, 236048, 276790, 290624, 301049, 318020, 335940, 353505, 361070, 370897]
roundEndedOfficial: [14830, 27154, 46544, 58496, 76416, 97677, 109787, 130834, 150026, 165973, 180737, 201878, 219050, 233488, 250588, 288064, 298489, 315460, 333380, 350945, 358510]
gameHalfEnded: [250332]
matchStart: []
matchStartedChanged: [0, 370897]
warmupChanged: [0]
teamSwitch: [250588]


In [ ]:
rounds = data["gameRounds"]
first_round = rounds[0]
for key in first_round.keys():
    print(key, type(first_round[key]))

In [ ]:
frames = first_round["frames"]
first_frame = frames[0]
for key in first_frame.keys():
    print(key, type(first_frame[key]))

In [3]:
import models

player_connections: list[models.PlayerConnection] = []
for connection in data["playerConnections"]:
    player_connections.append(
        models.PlayerConnection(
            tick=player["tick"],
            action=player["action"],
            steam_id=player["steamID"],
        )
    )

game_rounds: list[models.Round] = []
for round in data["gameRounds"]:
    ct_side: models.Team
    t_side: models.Team

    ct_players: list[models.Player] = []
    for player in round["ctSide"]["players"]:
        ct_players.append(models.Player(
            player_name=player["playerName"],
            steam_id=player["steamID"],
        ))
    ct_side = models.Team(
        team_name=round["ctSide"]["teamName"],
        players=ct_players,
    )

    t_players: list[models.Player] = []
    for player in round["tSide"]["players"]:
        t_players.append(models.Player(
            player_name=player["playerName"],
            steam_id=player["steamID"],
        ))
    t_side = models.Team(
        team_name=round["tSide"]["teamName"],
        players=t_players,
    )

    kills: list[models.Kill] = []
    for kill in round["kills"]:
        kills.append(models.Kill(
            tick=kill["tick"],
            seconds=kill["seconds"],
            clock_time=kill["clockTime"],
            attacker_steam_id=kill["attackerSteamID"],
            attacker_name=kill["attackerName"],
            attacker_team=kill["attackerTeam"],
            attacker_side=kill["attackerSide"],
            attacker_x=kill["attackerX"],
            attacker_y=kill["attackerY"],
            attacker_z=kill["attackerZ"],
            attacker_view_x=kill["attackerViewX"],
            attacker_view_y=kill["attackerViewY"],
            victim_steam_id=kill["victimSteamID"],
            victim_name=kill["victimName"],
            victim_team=kill["victimTeam"],
            victim_side=kill["victimSide"],
            victim_x=kill["victimX"],
            victim_y=kill["victimY"],
            victim_z=kill["victimZ"],
            victim_view_x=kill["victimViewX"],
            victim_view_y=kill["victimViewY"],
            assister_steam_id=kill["assisterSteamID"],
            assister_name=kill["assisterName"],
            assister_team=kill["assisterTeam"],
            assister_side=kill["assisterSide"],
            is_suicide=kill["isSuicide"],
            is_teamkill=kill["isTeamkill"],
            is_wallbang=kill["isWallbang"],
            penetrated_objects=kill["penetratedObjects"],
            is_first_kill=kill["isFirstKill"],
            is_headshot=kill["isHeadshot"],
            is_victim_blinded=kill["victimBlinded"],
            is_attacker_blinded=kill["attackerBlinded"],
            flash_thrower_steam_id=kill["flashThrowerSteamID"],
            flash_thrower_name=kill["flashThrowerName"],
            flash_thrower_team=kill["flashThrowerTeam"],
            flash_thrower_side=kill["flashThrowerSide"],
            is_no_scope=kill["noScope"],
            is_through_smoke=kill["thruSmoke"],
            distance=kill["distance"],
            is_trade=kill["isTrade"],
            player_traded_name=kill["playerTradedName"],
            player_traded_team=kill["playerTradedTeam"],
            player_traded_steam_id=kill["playerTradedSteamID"],
            weapon=kill["weapon"],
            weapon_class=kill["weaponClass"],
        ))

    damages: list[models.Damage] = []
    for damage in round["damages"]:
        damages.append(models.Damage(
            tick=damage["tick"],
            seconds=damage["seconds"],
            clock_time=damage["clockTime"],
            attacker_steam_id=damage["attackerSteamID"],
            attacker_name=damage["attackerName"],
            attacker_team=damage["attackerTeam"],
            attacker_side=damage["attackerSide"],
            attacker_x=damage["attackerX"],
            attacker_y=damage["attackerY"],
            attacker_z=damage["attackerZ"],
            attacker_view_x=damage["attackerViewX"],
            attacker_view_y=damage["attackerViewY"],
            is_attacker_strafe=damage["attackerStrafe"],
            victim_steam_id=damage["victimSteamID"],
            victim_name=damage["victimName"],
            victim_team=damage["victimTeam"],
            victim_side=damage["victimSide"],
            victim_x=damage["victimX"],
            victim_y=damage["victimY"],
            victim_z=damage["victimZ"],
            victim_view_x=damage["victimViewX"],
            victim_view_y=damage["victimViewY"],
            weapon=damage["weapon"],
            weapon_class=damage["weaponClass"],
            hp_damage=damage["hpDamage"],
            hp_damage_taken=damage["hpDamageTaken"],
            armor_damage=damage["armorDamage"],
            armor_damage_taken=damage["armorDamageTaken"],
            hit_group=damage["hitGroup"],
            is_friendly_fire=damage["isFriendlyFire"],
            distance=damage["distance"],
            zoom_level=damage["zoomLevel"],
        ))

    grenades: list[models.Grenade] = []
    for grenade in round["grenades"]:
        grenades.append(models.Grenade(
            throw_tick=grenade["throwTick"],
            destroy_tick=grenade["destroyTick"],
            throw_seconds=grenade["throwSeconds"],
            throw_clock_time=grenade["throwClockTime"],
            destroy_seconds=grenade["destroySeconds"],
            destroy_clock_time=grenade["destroyClockTime"],
            thrower_steam_id=grenade["throwerSteamID"],
            thrower_name=grenade["throwerName"],
            thrower_team=grenade["throwerTeam"],
            thrower_side=grenade["throwerSide"],
            thrower_x=grenade["throwerX"],
            thrower_y=grenade["throwerY"],
            thrower_z=grenade["throwerZ"],
            grenade_type=grenade["grenadeType"],
            grenade_x=grenade["grenadeX"],
            grenade_y=grenade["grenadeY"],
            grenade_z=grenade["grenadeZ"],
            entity_id=grenade["entityId"],
        ))

    bomb_events: list[models.BombEvent] = []
    for bomb_event in round["bombEvents"]:
        bomb_events.append(models.BombEvent(
            tick=bomb_event["tick"],
            seconds=bomb_event["seconds"],
            clock_time=bomb_event["clockTime"],
            player_steam_id=bomb_event["playerSteamID"],
            player_name=bomb_event["playerName"],
            player_team=bomb_event["playerTeam"],
            player_x=bomb_event["playerX"],
            player_y=bomb_event["playerY"],
            player_z=bomb_event["playerZ"],
            bomb_action=bomb_event["bombAction"],
            bomb_site=bomb_event["bombSite"],
        ))

    weapon_fires: list[models.WeaponFire] = []
    for weapon_fire in round["weaponFires"]:
        weapon_fires.append(models.WeaponFire(
            tick=weapon_fire["tick"],
            seconds=weapon_fire["seconds"],
            clock_time=weapon_fire["clockTime"],
            player_steam_id=weapon_fire["playerSteamID"],
            player_name=weapon_fire["playerName"],
            player_team=weapon_fire["playerTeam"],
            player_side=weapon_fire["playerSide"],
            player_x=weapon_fire["playerX"],
            player_y=weapon_fire["playerY"],
            player_z=weapon_fire["playerZ"],
            player_view_x=weapon_fire["playerViewX"],
            player_view_y=weapon_fire["playerViewY"],
            is_player_strafe=weapon_fire["playerStrafe"],
            weapon=weapon_fire["weapon"],
            weapon_class=weapon_fire["weaponClass"],
            ammo_in_magazine=weapon_fire["ammoInMagazine"],
            ammo_in_reserve=weapon_fire["ammoInReserve"],
            zoom_level=weapon_fire["zoomLevel"],
        ))

    flashes: list[models.Flash] = []
    for flash in round["flashes"]:
        flashes.append(models.Flash(
            tick=flash["tick"],
            seconds=flash["seconds"],
            clock_time=flash["clockTime"],
            attacker_steam_id=flash["attackerSteamID"],
            attacker_name=flash["attackerName"],
            attacker_team=flash["attackerTeam"],
            attacker_side=flash["attackerSide"],
            attacker_x=flash["attackerX"],
            attacker_y=flash["attackerY"],
            attacker_z=flash["attackerZ"],
            attacker_view_x=flash["attackerViewX"],
            attacker_view_y=flash["attackerViewY"],
            player_steam_id=flash["playerSteamID"],
            player_name=flash["playerName"],
            player_team=flash["playerTeam"],
            player_side=flash["playerSide"],
            player_x=flash["playerX"],
            player_y=flash["playerY"],
            player_z=flash["playerZ"],
            player_view_x=flash["playerViewX"],
            player_view_y=flash["playerViewY"],
            flash_duration=flash["flashDuration"],
        ))

    frames: list[models.Frame] = []
    for frame in round["frames"]:
        t: models.TeamFrameState
        
        t_players: list[models.PlayerFrameState] = []
        for player in frame["t"]["players"]:
            inventory: list[models.Weapon] = []
            for weapon in player["inventory"]:
                inventory.append(models.Weapon(
                    weapon_name=weapon["weaponName"],
                    weapon_class=weapon["weaponClass"],
                    ammo_in_magazine=weapon["ammoInMagazine"],
                    ammo_in_reserve=weapon["ammoInReserve"],
                ))

            t_players.append(models.PlayerFrameState(
                steam_id=player["steamID"],
                name=player["name"],
                team=player["team"],
                side=player["side"],
                x=player["x"],
                y=player["y"],
                z=player["z"],
                velocity_x=player["velocityX"],
                velocity_y=player["velocityY"],
                velocity_z=player["velocityZ"],
                view_x=player["viewX"],
                view_y=player["viewY"],
                hp=player["hp"],
                armor=player["armor"],
                active_weapon=player["activeWeapon"],
                total_utility=player["totalUtility"],
                is_alive=player["isAlive"],
                is_blinded=player["isBlinded"],
                is_airborne=player["isAirborne"],
                is_ducking=player["isDucking"],
                is_ducking_in_progress=player["isDuckingInProgress"],
                is_unducking_in_progress=player["isUnDuckingInProgress"],
                is_defusing=player["isDefusing"],
                is_planting=player["isPlanting"],
                is_reloading=player["isReloading"],
                is_in_bomb_zone=player["isInBombZone"],
                is_in_buy_zone=player["isInBuyZone"],
                is_standing=player["isStanding"],
                is_scoped=player["isScoped"],
                is_walking=player["isWalking"],
                is_unknown=player["isUnknown"],
                inventory=inventory,
                spotters=player["spotters"],
                equipment_value=player["equipmentValue"],
                equipment_value_freeze_time_end=player["equipmentValueFreezetimeEnd"],
                equipment_value_round_start=player["equipmentValueRoundStart"],
                cash=player["cash"],
                cash_spend_this_round=player["cashSpendThisRound"],
                cash_spend_total=player["cashSpendTotal"],
                has_helmet=player["hasHelmet"],
                has_defuse_kit=player["hasDefuse"],
                has_bomb=player["hasBomb"],
                ping=player["ping"],
                zoom_level=player["zoomLevel"],
            ))

        t = models.TeamFrameState(
            side=frame["t"]["side"],
            team_name=frame["t"]["teamName"],
            team_eq_val=frame["t"]["teamEqVal"],
            alive_players=frame["t"]["alivePlayers"],
            total_utility=frame["t"]["totalUtility"],
            players=t_players,
        )

        ct: models.TeamFrameState
        
        ct_players: list[models.PlayerFrameState] = []
        for player in frame["ct"]["players"]:
            inventory: list[models.Weapon] = []
            for weapon in player["inventory"]:
                inventory.append(models.Weapon(
                    weapon_name=weapon["weaponName"],
                    weapon_class=weapon["weaponClass"],
                    ammo_in_magazine=weapon["ammoInMagazine"],
                    ammo_in_reserve=weapon["ammoInReserve"],
                ))

            ct_players.append(models.PlayerFrameState(
                steam_id=player["steamID"],
                name=player["name"],
                team=player["team"],
                side=player["side"],
                x=player["x"],
                y=player["y"],
                z=player["z"],
                velocity_x=player["velocityX"],
                velocity_y=player["velocityY"],
                velocity_z=player["velocityZ"],
                view_x=player["viewX"],
                view_y=player["viewY"],
                hp=player["hp"],
                armor=player["armor"],
                active_weapon=player["activeWeapon"],
                total_utility=player["totalUtility"],
                is_alive=player["isAlive"],
                is_blinded=player["isBlinded"],
                is_airborne=player["isAirborne"],
                is_ducking=player["isDucking"],
                is_ducking_in_progress=player["isDuckingInProgress"],
                is_unducking_in_progress=player["isUnDuckingInProgress"],
                is_defusing=player["isDefusing"],
                is_planting=player["isPlanting"],
                is_reloading=player["isReloading"],
                is_in_bomb_zone=player["isInBombZone"],
                is_in_buy_zone=player["isInBuyZone"],
                is_standing=player["isStanding"],
                is_scoped=player["isScoped"],
                is_walking=player["isWalking"],
                is_unknown=player["isUnknown"],
                inventory=inventory,
                spotters=player["spotters"],
                equipment_value=player["equipmentValue"],
                equipment_value_freeze_time_end=player["equipmentValueFreezetimeEnd"],
                equipment_value_round_start=player["equipmentValueRoundStart"],
                cash=player["cash"],
                cash_spend_this_round=player["cashSpendThisRound"],
                cash_spend_total=player["cashSpendTotal"],
                has_helmet=player["hasHelmet"],
                has_defuse_kit=player["hasDefuse"],
                has_bomb=player["hasBomb"],
                ping=player["ping"],
                zoom_level=player["zoomLevel"],
            ))

        ct = models.TeamFrameState(
            side=frame["ct"]["side"],
            team_name=frame["ct"]["teamName"],
            team_eq_val=frame["ct"]["teamEqVal"],
            alive_players=frame["ct"]["alivePlayers"],
            total_utility=frame["ct"]["totalUtility"],
            players=t_players,
        )

        projectiles: list[models.Projectile] = []
        for projectile in frame["projectiles"]:
            projectiles.append(models.Projectile(
                projectile_type=projectile["projectileType"],
                x=projectile["x"],
                y=projectile["y"],
                z=projectile["z"],
            ))

        smokes: list[models.Smoke] = []
        for smoke in frame["smokes"]:
            smokes.append(models.Smoke(
                grenade_entity_id=smoke["grenadeEntityID"],
                start_tick=smoke["startTick"],
                x=smoke["x"],
                y=smoke["y"],
                z=smoke["z"],
            ))

        fires: list[models.Fire] = []
        for fire in frame["fires"]:
            fires.append(models.Fire(
                unique_id=fire["uniqueID"],
                x=fire["x"],
                y=fire["y"],
                z=fire["z"],
            ))

        frames.append(models.Frame(
            is_kill_frame = frame["isKillFrame"],
            tick=frame["tick"],
            seconds=frame["seconds"],
            clock_time=frame["clockTime"],
            t=t,
            ct=ct,
            bomb_planted=frame["bombPlanted"],
            bomb_site=frame["bombsite"],
            bomb=models.Bomb(
                x=frame["bomb"]["x"],
                y=frame["bomb"]["y"],
                z=frame["bomb"]["z"],
            ),
            projectiles=projectiles,
            smokes=smokes,
            fires=fires
        ))

    game_rounds.append(models.Round(
        round_number=round["roundNum"],
        is_warmup=round["isWarmup"],
        start_tick=round["startTick"],
        freeze_time_end_tick=round["freezeTimeEndTick"],
        end_tick=round["endTick"],
        end_official_tick=round["endOfficialTick"],
        bomb_plant_tick=round["bombPlantTick"],
        t_score=round["tScore"],
        ct_score=round["ctScore"],
        end_t_score=round["endTScore"],
        end_ct_score=round["endCTScore"],
        ct_team=round["ctTeam"],
        t_team=round["tTeam"],
        winning_side=round["winningSide"],
        winning_team=round["winningTeam"],
        losing_team=round["losingTeam"],
        round_end_reason=round["roundEndReason"],
        ct_freeze_time_end_eq_val=round["ctFreezeTimeEndEqVal"],
        ct_round_start_eq_val=round["ctRoundStartEqVal"],
        ct_round_spend_money=round["ctRoundSpendMoney"],
        ct_buy_type=round["ctBuyType"],
        t_freeze_time_end_eq_val=round["tFreezeTimeEndEqVal"],
        t_round_start_eq_val=round["tRoundStartEqVal"],
        t_round_spend_money=round["tRoundSpendMoney"],
        t_buy_type=round["tBuyType"],
        ct_side=ct_side,
        t_side=t_side,
        kills=kills,
        damages=damages,
        grenades=grenades,
        bomb_events=bomb_events,
        weapon_fires=weapon_fires,
        flashes=flashes,
        frames=frames,
    ))

structured_demo = models.Demo(
    match_id=data["matchID"],
    client_name=data["clientName"],
    map_name=data["mapName"],
    tick_rate=data["tickRate"],
    playback_ticks=data["playbackTicks"],
    playback_frames_count=data["playbackFramesCount"],
    parsed_to_frame_index=data["parsedToFrameIdx"],
    parser_parameters=models.ParserParameters(
        parse_rate=data["parserParameters"]["parseRate"],
        do_parse_frames=data["parserParameters"]["parseFrames"],
        do_parse_kill_frames=data["parserParameters"]["parseKillFrames"],
        trade_time=data["parserParameters"]["tradeTime"],
        round_buy_style=data["parserParameters"]["roundBuyStyle"],
        are_damages_rolled_up=data["parserParameters"]["damagesRolledUp"],
    ),
    server_vars=models.ServerVars(
        cash_bomb_defused=data["serverVars"]["cashBombDefused"],
        cash_bomb_planted=data["serverVars"]["cashBombPlanted"],
        cash_team_t_win_bomb=data["serverVars"]["cashTeamTWinBomb"],
        cash_win_defuse=data["serverVars"]["cashWinDefuse"],
        cash_win_time_run_out=data["serverVars"]["cashWinTimeRunOut"],
        cash_win_elimination=data["serverVars"]["cashWinElimination"],
        cash_player_killed_default=data["serverVars"]["cashPlayerKilledDefault"],
        cash_team_loser_bonus=data["serverVars"]["cashTeamLoserBonus"],
        cash_team_loser_bonus_consecutive=data["serverVars"]["cashTeamLoserBonusConsecutive"],
        round_time=data["serverVars"]["roundTime"],
        round_time_defuse=data["serverVars"]["roundTimeDefuse"],
        round_restart_delay=data["serverVars"]["roundRestartDelay"],
        freeze_time=data["serverVars"]["freezeTime"],
        buy_time=data["serverVars"]["buyTime"],
        bomb_timer=data["serverVars"]["bombTimer"],
        max_rounds=data["serverVars"]["maxRounds"],
        timeouts_allowed=data["serverVars"]["timeoutsAllowed"],
        coaching_allowed=data["serverVars"]["coachingAllowed"],
    ),
    match_phases=models.MatchPhases(
        announcement_last_round_half=data["matchPhases"]["announcementLastRoundHalf"],
        announcement_final_round=data["matchPhases"]["announcementFinalRound"],
        announcement_match_started=data["matchPhases"]["announcementMatchStarted"],
        round_started=data["matchPhases"]["roundStarted"],
        round_ended=data["matchPhases"]["roundEnded"],
        round_freeze_time_ended=data["matchPhases"]["roundFreezetimeEnded"],
        round_ended_official=data["matchPhases"]["roundEndedOfficial"],
        game_half_ended=data["matchPhases"]["gameHalfEnded"],
        match_start=data["matchPhases"]["matchStart"],
        match_started_changed=data["matchPhases"]["matchStartedChanged"],
        warmup_changed=data["matchPhases"]["warmupChanged"],
        team_switch=data["matchPhases"]["teamSwitch"],
    ),
    matchmaking_ranks=data["matchmakingRanks"],
    player_connections=player_connections,
    game_rounds=game_rounds,
)

TypeError: __init__() takes exactly 1 positional argument (2 given)